In [ ]:
# Dependencies
# %matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import time
from datetime import datetime, timedelta, date
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_columns', None)



In [ ]:
#diable warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Initially, we have a raw input file which is slightly different from the later version
#becuz Canada changed its content in reporting
#and after Canada changed it, it switched to all dates weekly
#however, we hope to keep the granularity of daily
#so we extract the columns that are the same in both versions
raw_input = pd.read_csv("daily_raw_input.csv")
#raw_input.head()

In [ ]:
cnd_raw = raw_input[(raw_input["Country"]=="Canada") & (raw_input["Level"]=="State/Province")]
#cnd_raw.head()

In [ ]:
#import both versions, use selectively
cnd_new = pd.read_csv("cnd_new_fx_0516.csv")
cnd_new_div7 = pd.read_csv("cnd_new_d7_fx_0516.csv")

In [ ]:
cnd_new_div7.info()

In [ ]:
common_cols = [col for col in cnd_new.columns if col in cnd_raw.columns]
print(common_cols)

In [ ]:
# loop through each row of the dataframe
for i, row in cnd_new.iterrows():
    # check if the Total Tests value is zero
    if row['Total Tests'] == 0:
        # loop backwards through the previous rows and find the last non-zero Total Tests value
        for j in range(i-1, -1, -1):
            if cnd_new.at[j, 'Total Tests'] != 0:
                # assign the last non-zero value to the current row
                cnd_new.at[i, 'Total Tests'] = cnd_new.at[j, 'Total Tests']
                break


In [ ]:
for i, row in cnd_new_div7.iterrows():
    # check if the Total Tests value is zero
    if row['Total Tests'] == 0:
        # loop backwards through the previous rows and find the last non-zero Total Tests value
        for j in range(i-1, -1, -1):
            if cnd_new_div7.at[j, 'Total Tests'] != 0:
                # assign the last non-zero value to the current row
                cnd_new_div7.at[i, 'Total Tests'] = cnd_new_div7.at[j, 'Total Tests']
                break

In [ ]:
cnd_new.to_csv("cnd_new_rep0.csv", index=False)
cnd_new_div7.to_csv("cnd_new_div7_rep0.csv", index=False)

In [ ]:
#just to see the latest dates, note that our "date" column is called Time!
cnd_new[cnd_new["State/Province"]=="Alberta"].tail(20)

In [ ]:
cnd_new[cnd_new["State/Province"]=="Alberta"].to_csv("albertatest.csv", index=False)

In [ ]:
cnd_raw['Time'] = pd.to_datetime(cnd_raw['Time'])
cnd_new['Time'] = pd.to_datetime(cnd_new['Time'])
cnd_new_div7['Time'] = pd.to_datetime(cnd_new_div7['Time'])

#here we cut the new version that have dates exceeding the raw version
nnd = cnd_new.loc[cnd_new['Time'] > cnd_raw['Time'].max()]
nnd

In [ ]:
n7d = cnd_new_div7.loc[cnd_new_div7['Time'] > cnd_raw['Time'].max()]
n7d

In [ ]:
cnd_raw

In [ ]:
rawcom = cnd_raw[common_cols]
rawcom

In [ ]:
nndcom = nnd[common_cols]
n7dcom = n7d[common_cols]

Now we have the common columns from both dataframes, we concatenate them together

In [ ]:
cnd_new_cb = pd.concat([rawcom, nndcom], axis=0)
cnd_new_div7_cb = pd.concat([rawcom, n7dcom], axis=0)

In [ ]:
cnd_new_cb.to_csv("cnd_new_cb.csv", index=False)
cnd_new_div7_cb.to_csv("cnd_new_div7_cb.csv", index=False)